In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [3]:
train_all=pd.read_csv('train_pre11.csv')

In [4]:
ft=np.logical_and(train_all.day.values>=17,train_all.day.values<=25)
train_Small = train_all.ix[ft, :]
train_Small.label.value_counts()

0    2183425
1      56764
Name: label, dtype: int64

In [5]:
np.random.seed(999)
r1 = np.random.uniform(0, 1, train_Small.shape[0])
train_X = train_Small.ix[r1 < 0.1, :]

In [6]:
test_Small= train_all.ix[train_all.day.values==25]
test_Small.shape[0]

266833

In [6]:
np.random.seed(999)
r1 = np.random.uniform(0, 1, test_Small.shape[0])
test_X = test_Small.ix[r1 < 0.5, :]

In [7]:
import math
def loss(label,pro):
    label1=label.values
    pro1=pro
    prob1=pro.apply(lambda x: math.log(x)).values
    prob2=pro1.apply(lambda x:math.log(1-x)).values
    Loss=-np.mean((label1*pro1)+(1-label1)*prob2)
    return Loss

In [8]:
-

In [9]:
target='label'
day='day'
id='instanceID'

In [12]:
predictors = [x for x in train_X.columns if x not in ['label','conversionTime'
                                         ,'camgaignID' ,'adID_advertiserID'  , 'creativeID_sitesetID' ,'creativeID','sitesetID','residence2'
                                        ,'creativeID_haveBaby','day', 'creativeID_gender','adID','adID_gender', 'haveBaby','appID'
                                        ,'creativeID_residence2','age','sitesetID_positionType','sitesetID_haveBaby', 'marriageStatus'
                                        ,'appPlatform_age','adID_haveBaby','camgaignID_haveBaby', 'telecomsOperator','appPlatform_edu','age_gender' ]]xgb1 = XGBClassifier(

xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=3,
        min_child_weight=5,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)

In [ ]:
modelfit(xgb1, train_X, test_X, predictors)

In [13]:
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_X[predictors],train_X[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.46803, std: 0.17174, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.47231, std: 0.17571, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.47694, std: 0.16818, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.42550, std: 0.20123, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.43467, std: 0.20077, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.43124, std: 0.19798, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.41264, std: 0.21041, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.41408, std: 0.20963, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.41893, std: 0.20947, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.40600, std: 0.20848, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.40829, std: 0.21287, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.41115, std: 0.21141, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 5

In [10]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test2 = {
    'max_depth':[3,4,5],
    'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_X[predictors],train_X[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

NameError: name 'train' is not defined

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test2b = {
    'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(train_X[predictors],train_X[target])
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train_X[predictors],train_X[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=4,
        min_child_weight=6,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb2, train, test, predictors)

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train_X[predictors],train_X[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test5 = {
    'subsample':[i/100.0 for i in range(75,90,5)],
    'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
                                        min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(train[predictors],train[target])
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
                                        min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(train[predictors],train[target])
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test7 = {
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
                                        min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(train[predictors],train[target])
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

In [ ]:
xgb3 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=4,
        min_child_weight=6,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.005,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb3, train, test, predictors)

In [ ]:
xgb4 = XGBClassifier(
        learning_rate =0.01,
        n_estimators=5000,
        max_depth=4,
        min_child_weight=6,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.005,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb4, train, test, predictors)